<a href="https://colab.research.google.com/github/JacksonLeb/tensorflow_learning/blob/main/08_rnn_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#introduction to NLP fundamentals in Tf

##Getting info from natural language (sequences or speech)
##NLP can also be called seq2seq problems

###Checking for GPU


In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-06508a3c-c558-cf41-39bb-afd2ff21b5e6)


In [2]:
##Get helper functions
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-07-24 20:35:49--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-07-24 20:35:49 (84.7 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [4]:
##Get a text dataset
###Using the Kaggle Intro to NLP dataset, samples of tweets labeled hazard or not
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2022-07-24 20:35:52--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.24.128, 172.217.194.128, 142.251.10.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.24.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2022-07-24 20:35:52 (92.0 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
unzip_data("nlp_getting_started.zip")

##Visualizing a text dataset

In [6]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
train_df["text"][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [8]:
train_df["text"][10]

'Three people died from the heat wave so far'

In [9]:
train_df["text"][100]

'.@NorwayMFA #Bahrain police had previously died in a road accident they were not killed by explosion https://t.co/gFJfgTodad'

In [10]:
#shuffle trianing df
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [11]:
#test df
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [12]:
#examples of each class
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [13]:
#total num of samples
len(train_df), len(test_df)

(7613, 3263)

In [14]:
#visualizing random samples
import random
random_index = random.randint(0, len(train_df)-5) #create random index
for row in train_df_shuffled[["text", "target"]][random_index:random_index +5 ].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}")

Target: 0 (not real disaster)
Text:
Homeless Man Traveling Across USA With 11 Stray Dogs He Rescued Gets A Helping Hand From Strangers http://t.co/QhfqlUI6RY via @Reshareworthy
Target: 0 (not real disaster)
Text:
He better than Sean bro. I can admit that Sean is flames now. But he better than Sean https://t.co/aomQ1RYKmJ
Target: 0 (not real disaster)
Text:
#veterans VET Act would ensure every military veteran's access to highest level of emergency care services: Proper treatmen...  #followme
Target: 0 (not real disaster)
Text:
@Annealiz1 You are going to make the internet explode with this Dr. Simon. O_o ... Was he alone or was there a red-head nearby? LOL
Target: 0 (not real disaster)
Text:
@local_arsonist LMFAO


In [17]:
#splittting data into training and validation
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=30)

In [18]:
#checking lengths of each data set
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

In [19]:
#check the first 10 sentences
train_sentences[:10]

array(["All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       "@DavidJordan88 @Stephanenny Except we don't know who started the riot or if it even makes sense to credit any particular individuals...",
       'Shadow boxing the apocalypse',
       "Maybe that's what happens when a tornado meets a volcano",
       '#Politics Democracy\x89Ûªs hatred for hate: \x89Û_ Dawabsha threaten to erode Israeli democracy. Homegrown terrorism ha...  http://t.co/q8n5Tn8WME',
       "@TomDean86 he's alright but Hazard/Willian ain't gonna be shifted easily.",
       '0-day bug in fully patched OS X comes under active exploit to hijack Macs http://t.co/sbGiRvQvzb',
       'Three Israeli soldiers wounded in West Bank terrorist attack via /r/worldnews http://t.co/su4ZVWADj7',
       'If it\x89Ûªs a war you came to see you will never see a waved white flag in front me.\nI can\x89Ûªt end up dead I wont be misled.',
     

In [20]:
val_sentences[:10]

array(['Blaze is my bro http://t.co/UdKeSJ01mL',
       'I was taught at school in the 1970s that piracy slavery and suicide-bombing were purely historical. No one then expected them to re-occur',
       'http://t.co/FueRk0gWui Twelve feared killed in Pakistani air ambulance helicopter crash http://t.co/Mv7GgGlmVc',
       "I'm a friggin wreck destiel sucks (read the vine description)\nhttps://t.co/MKX6Ux4OZt",
       'Dr. Bengston on #wildfire management: \x89ÛÏnumbers and size of fires areas affected and costs of fighting them all show upward trend.\x89Û\x9d #smem',
       '@ameenshaikh3 sir i just only wanted to make a point about @sureshpprabhu you made and said he is lying about bridge collapse.',
       'Wow! I just won this for free The Hobbit: Desolation of Smaug UV digital download code *GIN 9 http://t.co/MjFdCrjs8j #listia',
       'Photoset: hakogaku: ?åÊI am a kurd. i was born on a battlefield. raised on a battlefield. gunfire sirens... http://t.co/obp595W7tm',
       'Wash

## Converting text into numbers

#Tokenization and Embedding
##Tokenization: Each word/character is set to a number/one-hot encoding
##Embedding: Each word/character is set to a vector, with user-defined size, allowing for information on the relationship between tokens/words/characters. Additionally, can limit size and can be learned (treated as a weight matrix)


In [25]:
#Text Vectorization
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

#Use default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, #how many words in vocabulary (automaticaly add <OOV>)
                                    standardize="lower_and_strip_punctuation", #remove fluff,
                                    split="whitespace",
                                    ngrams=None, #create groups of n words
                                    output_mode="int", #how to map tokens to numbers
                                    output_sequence_length=None, #how long each batch of a setence should be
                                    pad_to_max_tokens=False)


In [27]:
#find average number of tokens (words) in the trianing tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [28]:
#set up text vectorization variables
max_vocab_length = 10000 # max num of words to have in vocabulary
max_length = 15 #max length of a tweet that our models sees

text_vectorizer = TextVectorization(max_tokens=max_vocab_length, #how many words in vocabulary (automaticaly add <OOV>)
                                    standardize="lower_and_strip_punctuation", #remove fluff,
                                    split="whitespace",
                                    ngrams=None, #create groups of n words
                                    output_mode="int", #how to map tokens to numbers
                                    output_sequence_length=max_length, #how long each batch of a setence should be
                                    pad_to_max_tokens=True)

In [29]:
#adapting text_vectorizer to training data
text_vectorizer.adapt(train_sentences) #adapt to training data, very important

In [30]:
train_sentences[:10]

array(["All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       "@DavidJordan88 @Stephanenny Except we don't know who started the riot or if it even makes sense to credit any particular individuals...",
       'Shadow boxing the apocalypse',
       "Maybe that's what happens when a tornado meets a volcano",
       '#Politics Democracy\x89Ûªs hatred for hate: \x89Û_ Dawabsha threaten to erode Israeli democracy. Homegrown terrorism ha...  http://t.co/q8n5Tn8WME',
       "@TomDean86 he's alright but Hazard/Willian ain't gonna be shifted easily.",
       '0-day bug in fully patched OS X comes under active exploit to hijack Macs http://t.co/sbGiRvQvzb',
       'Three Israeli soldiers wounded in West Bank terrorist attack via /r/worldnews http://t.co/su4ZVWADj7',
       'If it\x89Ûªs a war you came to see you will never see a waved white flag in front me.\nI can\x89Ûªt end up dead I wont be misled.',
     

In [31]:
#Create sample setnces and tokenize it
sample_sentence = "A tornado destroyed my house"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  3, 595, 346,  13, 311,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [32]:
#Choose a random sentence from the triaing dataset and vectorize it
random_sentence = random.choice(train_sentences)
print(f"original sentence: {random_sentence}\n vectorized sentence: {text_vectorizer([random_sentence])}")

original sentence: @DauntedPsyche - Man gently dabbed the cotton rag across each one of his wounds; the pain a lot more worse than Glenn had ever -
 vectorized sentence: [[   1   90    1    1    2 3089 9389  821 1537   58    6   80  394    2
   803]]


In [35]:
#number of unique words found in train_sentences
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Num of words: {len(words_in_vocab)}")
top_5_words, bottom_5_words

Num of words: 10000


(['', '[UNK]', 'the', 'a', 'in'],
 ['panda', 'panama', 'pampalmater', 'palmoil', 'palmer'])

### Creating an embedding using an embedding layer


In [36]:
#using tensorflow.keras.layeres.Embedding
##Parameters: input_dim, output_dim, input_length
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             input_length=max_length)
embedding

In [38]:
#get a random sentence
random_sentence = random.choice(train_sentences)
print(f"random sentence: {random_sentence}")
print(f"embedded:")
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

random sentence: Afghan Soldier Kills US General America's Highest-Ranking Fatality Since Vietnam http://t.co/SiHQPlUIDW
embedded:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.04714501, -0.00964833, -0.01940884, ..., -0.03806298,
         -0.0076165 ,  0.04934535],
        [-0.01310224, -0.04060646, -0.0386183 , ..., -0.00948709,
          0.03057035,  0.03662935],
        [ 0.03338554,  0.02873928,  0.01231179, ...,  0.02865131,
         -0.01105748,  0.01282597],
        ...,
        [ 0.04302993, -0.01552999, -0.02750639, ..., -0.04369334,
         -0.02614435, -0.012084  ],
        [ 0.04302993, -0.01552999, -0.02750639, ..., -0.04369334,
         -0.02614435, -0.012084  ],
        [ 0.04302993, -0.01552999, -0.02750639, ..., -0.04369334,
         -0.02614435, -0.012084  ]]], dtype=float32)>

In [39]:
sample_embed[0][0], sample_embed[0][0].shape, 

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.04714501, -0.00964833, -0.01940884,  0.00151864,  0.03490773,
         0.00971832, -0.01224781,  0.00119209, -0.01944307, -0.03019482,
        -0.01551268, -0.01103187,  0.04268459, -0.02275138,  0.04717778,
        -0.00801599, -0.01387594,  0.02674618, -0.02029307,  0.04703662,
         0.04171001, -0.02575157,  0.03916701,  0.00615125,  0.02667112,
         0.04812009,  0.02643652, -0.01358604, -0.04232427,  0.02661045,
        -0.00013716, -0.0434395 ,  0.04982186, -0.04882513,  0.01250983,
         0.03634072,  0.03329799, -0.01556236, -0.01401138,  0.03687742,
        -0.01773504, -0.02894297,  0.02668447, -0.01806512,  0.02593673,
        -0.04480617, -0.01542222, -0.00941701, -0.02753955,  0.02918718,
         0.04162736, -0.04851066,  0.03679479, -0.00930024,  0.00845813,
        -0.04113225, -0.00137137, -0.02294987, -0.0326397 , -0.03745742,
         0.0125229 ,  0.02162289,  0.01144407,  0.01665261,  0.00974159,
  

##Modelling a text dataset (running series of experiments)
###1. Naive bayes with Tf-IDF encoder (BASELINE)
###2. Feed-forward neural network 
###3. LSTM
###4. GRU
###5. Bidirectiona;-LSTM
###6. 1D CNN
###7. TFHub pretrianed feature extractor
###8. TFHub pretrained with 10% of trained datam

